<a href="https://colab.research.google.com/github/shin0729/Spatial-Lecture-on-NLP-implementation/blob/main/Untitled13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/shavarani/SpEL.git
%cd /content/SpEL/SpEL/src/spel


Cloning into 'SpEL'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 421 (delta 116), reused 109 (delta 75), pack-reused 251
Receiving objects: 100% (421/421), 33.22 MiB | 13.95 MiB/s, done.
Resolving deltas: 100% (221/221), done.
/content/SpEL/SpEL/src/spel


In [ ]:
!pip install -r requirements.txt


In [ ]:
import sys
sys.path.append('/content/SpEL/src')
!wget -O model.pt https://vault.sfu.ca/index.php/s/kBBlYVM4Tr59P0q/download



--2024-05-22 11:56:24--  https://vault.sfu.ca/index.php/s/kBBlYVM4Tr59P0q/download
Resolving vault.sfu.ca (vault.sfu.ca)... 142.58.225.42
Connecting to vault.sfu.ca (vault.sfu.ca)|142.58.225.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1444752929 (1.3G) [application/octet-stream]
Saving to: ‘model.pt’

model.pt            100%[===================>]   1.34G   666KB/s    in 36m 18s 

2024-05-22 12:32:43 (648 KB/s) - ‘model.pt’ saved [1444752929/1444752929]



In [ ]:
!ls /content/SpEL/src/spel


aida.py		      data_loader.py	  finetune_step_2.py  server.py
base_model.cfg	      decao_eval.py	  finetune_step_3.py  span_annotation.py
candidate_manager.py  evaluate_local.py   model.py	      utils.py
configuration.py      finetune_step_1.py  __pycache__	      visualize.py


In [ ]:
from transformers import AutoTokenizer
from spel.model import SpELAnnotator
from spel.configuration import device
from spel.utils import get_subword_to_word_mapping
from spel.span_annotation import WordAnnotation, PhraseAnnotation
finetuned_after_step = 4
sentence = "Natural Language Processing (NLP) is a subfield of artificial intelligence (AI) focused on the interaction between computers and human language. It involves the application of computational techniques to analyze and synthesize natural language and speech. Key tasks in NLP include language modeling, syntactic parsing, machine translation, sentiment analysis, and named entity recognition. Recent advancements in deep learning, particularly transformer-based models like BERT and GPT, have significantly improved the performance of NLP systems, enabling more accurate and sophisticated language understanding and generation."
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# ############################################# LOAD SpEL #############################################################
spel = SpELAnnotator()
spel.init_model_from_scratch(device=device)
if finetuned_after_step == 3:
    spel.shrink_classification_head_to_aida(device)
spel.load_checkpoint(None, device=device, load_from_torch_hub=True, finetuned_after_step=finetuned_after_step)
# ############################################# RUN SpEL ##############################################################
inputs = tokenizer(sentence, return_tensors="pt")
token_offsets = list(zip(inputs.encodings[0].tokens,inputs.encodings[0].offsets))
subword_annotations = spel.annotate_subword_ids(inputs.input_ids, k_for_top_k_to_keep=10, token_offsets=token_offsets)
# #################################### CREATE WORD-LEVEL ANNOTATIONS ##################################################
tokens_offsets = token_offsets[1:-1]
subword_annotations = subword_annotations[1:]
word_annotations = [WordAnnotation(subword_annotations[m[0]:m[1]], tokens_offsets[m[0]:m[1]])
                    for m in get_subword_to_word_mapping(inputs.tokens(), sentence)]
# ################################## CREATE PHRASE-LEVEL ANNOTATIONS ##################################################
phrase_annotations = []
for w in word_annotations:
    if not w.annotations:
        continue
    if phrase_annotations and phrase_annotations[-1].resolved_annotation == w.resolved_annotation:
        phrase_annotations[-1].add(w)
    else:
        phrase_annotations.append(PhraseAnnotation(w))


 * Loaded pretrained model checkpoint: spel-base-step-3-500K.pt
 * Loaded model with 511844697 number of parameters (124697433 parameters for the encoder and 387147264 parameters for the classification head)!


In [ ]:
# Word-Level Annotations
print("Word-Level Annotations:")
for word_annotation in word_annotations:
    print(f"Word: {word_annotation.word_string}, Annotation: {word_annotation.resolved_annotation}")

# Phrase-Level Annotations
print("\nPhrase-Level Annotations:")
for phrase_annotation in phrase_annotations:
    phrase_text = ' '.join([word.word_string for word in phrase_annotation.words])
    print(f"Phrase: {phrase_text}, Annotation: {phrase_annotation._resolved_annotation}")

Word-Level Annotations:
Word: Natural, Annotation: 0
Word: Language, Annotation: 0
Word: Processing, Annotation: 0
Word: (, Annotation: 0
Word: NLP, Annotation: 0
Word: ), Annotation: 0
Word: is, Annotation: 0
Word: a, Annotation: 0
Word: subfield, Annotation: 0
Word: of, Annotation: 0
Word: artificial, Annotation: 21982
Word: intelligence, Annotation: 21982
Word: (, Annotation: 0
Word: AI, Annotation: 0
Word: ), Annotation: 0
Word: focused, Annotation: 0
Word: on, Annotation: 0
Word: the, Annotation: 0
Word: interaction, Annotation: 0
Word: between, Annotation: 0
Word: computers, Annotation: 12180
Word: and, Annotation: 0
Word: human, Annotation: 0
Word: language, Annotation: 0
Word: ., Annotation: 0
Word: It, Annotation: 0
Word: involves, Annotation: 0
Word: the, Annotation: 0
Word: application, Annotation: 0
Word: of, Annotation: 0
Word: computational, Annotation: 0
Word: techniques, Annotation: 0
Word: to, Annotation: 0
Word: analyze, Annotation: 0
Word: and, Annotation: 0
Word: sy